# FMRI Data Exploration


## StarPlus fMRI data

* Functional Magnetic Resonance Imaging (fMRI) is a technique for obtaining three-dimensional images related to neural 
activity in the brain through time. 
* It measures the ratio of oxygenated haemoglobin to deoxygenated haemoglobin in the blood with respect to a control
baseline, at many individual locations with the brain.
* It is widely believed that blood oxygen level is influenced by local neural activity, and hence this blood oxygen
level dependent (BOLD) response is generally taken as an indicator of neural activity. 
* An fMRI scanner measures the value of the fMRI signal (BOLD response) at all points in a three dimensional grid, or
image.
* We refer to cells within this 3D image as *voxels* (volumetric elements of pixels).
* The voxels in a typical fMRI study has a volume of a few tens of cubic millimeters, and a typical 3D brain image
typically contains 10,000 to 15,000 voxels which contain cortical matter and are thus of interest. 
* 

In [1]:
# imports

import scipy.io as sio
import numpy as np


### Notes about StarPlus dataset:
- Each data file is for a subject with the subject id in as the integer in the title
- The files are marked to be loaded in Matlab 7


In [2]:
test = sio.loadmat('data/data-starplus-04847-v7.mat')

nd_info = test['info']
nd_data = test['data']

print('Mat Structure')
sio.whosmat('data/data-starplus-04847-v7.mat')


Mat Structure


[('info', (1, 54), 'struct'),
 ('data', (54, 1), 'cell'),
 ('meta', (1, 1), 'struct')]

In [45]:
print(test['info'].__class__)
print(test['info'].dtype)
print(test['data'].__class__)
print(test['data'].dtype)
print(test['meta'].__class__)
print(test['meta'].dtype)
print(nd_info.shape)
print(nd_data.shape)


<class 'numpy.ndarray'>
[('cond', 'O'), ('len', 'O'), ('mint', 'O'), ('maxt', 'O'), ('missing', 'O'), ('nmissing', 'O'), ('sentence', 'O'), ('img', 'O'), ('sentenceRel', 'O'), ('sentenceSym1', 'O'), ('sentenceSym2', 'O'), ('actionRT', 'O'), ('actionAnswer', 'O'), ('firstStimulus', 'O')]
<class 'numpy.ndarray'>
object
<class 'numpy.ndarray'>
[('colToCoord', 'O'), ('coordToCol', 'O'), ('nvoxels', 'O'), ('dimx', 'O'), ('dimy', 'O'), ('dimz', 'O'), ('ntrials', 'O'), ('study', 'O'), ('testReadingSpan', 'O'), ('testVandenburg', 'O'), ('testPaperFold', 'O'), ('testSurfaceFold', 'O'), ('subject', 'O'), ('roi', 'O'), ('nsnapshots', 'O'), ('rois', 'O'), ('colToROI', 'O')]
(1, 54)
(54, 1)


In [43]:
nd_info.dtype


dtype([('cond', 'O'), ('len', 'O'), ('mint', 'O'), ('maxt', 'O'), ('missing', 'O'), ('nmissing', 'O'), ('sentence', 'O'), ('img', 'O'), ('sentenceRel', 'O'), ('sentenceSym1', 'O'), ('sentenceSym2', 'O'), ('actionRT', 'O'), ('actionAnswer', 'O'), ('firstStimulus', 'O')])

In [74]:
print(nd_info[0].shape)
print(nd_info[0][0][0].shape)
nd_info[0][0][7]

(54,)
(1, 1)


array([], dtype='<U1')

In [57]:
print(nd_data[0].shape)
print(nd_data[0][0].shape)
print(nd_data[0][0][0].shape)
print(nd_data[0][0][0][0].shape)


(1,)
(13, 4698)
(4698,)
()


## References

[1] 
